In [4]:
import random
import psycopg2
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Database connection setup
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")

# Establish the connection using psycopg2
conn = psycopg2.connect(
    dbname=db_name, user=db_user, password=db_password, host=db_host
)
print(f"Connected to database: {db_name}")

def get_cash_balance(conn, account_id):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT total_cash_balance
            FROM public.vw_cash_balance
            WHERE accountid = %s
            ORDER BY total_cash_balance DESC
            LIMIT 1;
        """, (account_id,))
        result = cur.fetchone()
    return result[0] if result else 0

def get_asset_list(conn):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT assetid, latestprice, currencycode
            FROM public.vw_latest_price
            WHERE assetid != 699;
        """)
        return cur.fetchall()

def simulate_trade(conn, account_id):
    try:
        # Get the cash balance for the account
        cash_balance = get_cash_balance(conn, account_id)

        # Only process if the cash balance is greater than 0
        if cash_balance <= 0:
            print(f"Account {account_id} has insufficient cash balance (cash balance: {cash_balance}). Skipping.")
            return

        # Retrieve the asset list (assets available for trade)
        assets = get_asset_list(conn)

        if not assets:
            print(f"No assets available for trading for account {account_id}.")
            return

        # Select a random asset to trade
        asset_id, asset_price, currency_code = random.choice(assets)

        # Explicitly convert to standard Python types (float for account_id and asset_id)
        account_id = float(account_id)
        asset_id = int(asset_id)

        # Debug print the cash balance and asset price
        print(f"Account {account_id} has cash balance: {cash_balance}, selected asset {asset_id} price: {asset_price}")

        # Determine the maximum quantity the user can afford to buy based on the cash balance
        max_quantity = cash_balance // asset_price

        # Debug print the max quantity
        print(f"Account {account_id} can afford {max_quantity} units of asset {asset_id}.")

        if max_quantity < 1:
            print(f"Account {account_id} does not have enough balance to buy any asset.")
            return

        # Randomly choose trade type: Buy or Sell
        trade_type = random.choice(['Buy', 'Sell'])

        # Buy Logic
        if trade_type == 'Buy':
            trade_quantity = random.randint(1, max_quantity)
            trade_cost = trade_quantity * asset_price

            # Check if account has enough cash to complete the buy trade
            if trade_cost > cash_balance:
                print(f"Account {account_id} does not have enough cash for the buy trade.")
                return

            # Insert buy trade into the trade table
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO public.trade (
                        accountid, assetid, tradequantity, tradecost, tradetype, tradestatus, datecreated, dateupdated
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (account_id, asset_id, trade_quantity, trade_cost, 'Buy', 'Completed', datetime.now(), datetime.now()))

            # Deduct the cost of the buy trade from the cash balance by inserting a "buy" for asset ID 699
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO public.trade (
                        accountid, assetid, tradequantity, tradecost, tradetype, tradestatus, datecreated, dateupdated
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (account_id, 699, trade_quantity, -trade_cost, 'Buy', 'Completed', datetime.now(), datetime.now()))

            conn.commit()

            print(f"Account {account_id} bought {trade_quantity} of asset {asset_id} at {asset_price} {currency_code}. Deducted {trade_cost} from cash.")

        # Sell Logic
        elif trade_type == 'Sell':
            # First, check how much of the asset the account owns
            with conn.cursor() as cur:
                cur.execute("""
                    SELECT SUM(tradequantity) as total_quantity
                    FROM public.trade
                    WHERE accountid = %s AND assetid = %s AND tradetype = 'Buy' AND datecreated < %s;
                """, (account_id, asset_id, datetime.now().date()))
                result = cur.fetchone()

            owned_quantity = result[0] or 0

            # Ensure the account has enough of the asset to sell
            if owned_quantity <= 0:
                print(f"Account {account_id} does not own any {asset_id} to sell.")
                return

            trade_quantity = random.randint(1, min(owned_quantity, max_quantity))  # Sell a random quantity, up to the maximum allowed
            trade_cost = trade_quantity * asset_price  # Proceeds from selling

            # Insert sell trade into the trade table
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO public.trade (
                        accountid, assetid, tradequantity, tradecost, tradetype, tradestatus, datecreated, dateupdated
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (account_id, asset_id, trade_quantity, trade_cost, 'Sell', 'Completed', datetime.now(), datetime.now()))

            # Add the proceeds from the sale to the cash balance by inserting a "sell" for asset ID 699
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO public.trade (
                        accountid, assetid, tradequantity, tradecost, tradetype, tradestatus, datecreated, dateupdated
                    )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (account_id, 699, trade_quantity, trade_cost, 'Sell', 'Completed', datetime.now(), datetime.now()))

            conn.commit()

            print(f"Account {account_id} sold {trade_quantity} of asset {asset_id} at {asset_price} {currency_code}. Added {trade_cost} to cash.")

    except Exception as e:
        print(f"Error simulating trade for account {account_id}: {e}")

# Example usage: Simulate trades for 1000 random accounts
for _ in range(10000):
    simulate_trade(conn, random.randint(80000, 200000))

conn.close()


Connected to database: postgres
Account 173143.0 has cash balance: 3000, selected asset 690 price: 106.37000
Account 173143.0 can afford 28 units of asset 690.
Account 173143.0 does not own any 690 to sell.
Account 162683.0 has cash balance: 3000, selected asset 670 price: 107.53000
Account 162683.0 can afford 27 units of asset 670.
Account 162683.0 does not own any 670 to sell.
Account 123084.0 has cash balance: 2000, selected asset 623 price: 17.82000
Account 123084.0 can afford 112 units of asset 623.
Account 123084.0 does not own any 623 to sell.
Account 116994.0 has cash balance: 1000, selected asset 691 price: 177.96271
Account 116994.0 can afford 5 units of asset 691.
Account 116994.0 does not own any 691 to sell.
Account 132825.0 has cash balance: 1000, selected asset 604 price: 423.41501
Account 132825.0 can afford 2 units of asset 604.
Account 132825.0 bought 2 of asset 604 at 423.41501 USD. Deducted 846.83002 from cash.
Account 96010.0 has cash balance: 3000, selected asset 

KeyboardInterrupt: 